# Create an Agent with a single Knowledge Base only

In this notebook you will learn how to create an Amazon Bedrock Agent that connects to a single Knowledge Bases for Amazon Bedrock to retrieve company data and complete tasks. 

The use case for this notebook is the Amazon Bedrock Documentation pages stored as PDFs. It will allow you to ask questions about Amazon Bedrock and get answers based on documents available in the Knowledge Base.

The steps to complete this notebook are:

1. Import the needed libraries
1. Create an S3 bucket and upload the data to it
1. Create the Knowledge Base for Amazon Bedrock and sync data to Knowledge Base
1. Create the Agent for Amazon Bedrock
1. Test the Agent
1. Clean up the resources created

## 1. Import the needed libraries

In [1]:
!pip install --upgrade -q opensearch-py
!pip install --upgrade -q requests-aws4auth
!pip install --upgrade -q boto3
!pip install --upgrade -q botocore
!pip install --upgrade -q awscli

In [2]:
import logging
import boto3
import time
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

[2025-02-09 15:26:44,216] p57121 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [5]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '174178623257')

In [6]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "bedrock-docs-kb-agents"
agent_alias_name = "bedrock-docs-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_arn = f"arn:aws:s3:::{bucket_name}"
bedrock_agent_bedrock_allow_policy_name = f"bda-bedrock-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"bda-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_bedrock_docs'
kb_name = f'bedrock-docs-kb-{suffix}'
data_source_name = f'bedrock-docs-kb-docs-{suffix}'
kb_files_path = 'kb_documents'
kb_key = 'kb_documents'
kb_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'
kb_bedrock_allow_policy_name = f"bd-kb-bedrock-allow-{suffix}"
kb_aoss_allow_policy_name = f"bd-kb-aoss-allow-{suffix}"
kb_s3_allow_policy_name = f"bd-kb-s3-allow-{suffix}"
kb_collection_name = f'bd-kbc-{suffix}'
# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1'
kb_vector_index_name = "bedrock-knowledge-base-index"
kb_metadataField = 'bedrock-knowledge-base-metadata'
kb_textField = 'bedrock-knowledge-base-text'
kb_vectorField = 'bedrock-knowledge-base-vector'
model_id = "amazon.nova-pro-v1:0"

# agent configuration
agent_instruction = """
You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Base
and you can Answer questions from this documentation. Only answer questions based on the documentation and reply with 
"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?" 
If the answer to the question is not available in the documentation
"""

## 2. Upload the dataset to Amazon S3
Knowledge Bases for Amazon Bedrock, currently require data to reside in an Amazon S3 bucket. In this section we will create an Amazon S3 bucket and the files.

### 2.1 Create the Amazon S3 bucket

In [7]:
if region != 'us-east-1':
    s3_client.create_bucket(
        Bucket=bucket_name.lower(),
        CreateBucketConfiguration={'LocationConstraint': region}
    )
else:
    s3_client.create_bucket(Bucket=bucket_name)

### 2.2 Upload dataset to the Amazon S3 bucket

In [8]:
# Upload Knowledge Base files to this s3 bucket
for f in os.listdir(kb_files_path):
    if f.endswith(".pdf"):
        s3_client.upload_file(kb_files_path+'/'+f, bucket_name, kb_key+'/'+f)

## 3. Create a Knowledge Base for Amazon Bedrock

In this section we will go through all the steps to create and test a Knowledge Base. 

These are the steps to complete:
    
1. Create a Knowledge Base Role and its policies
1. Create a Vector Database
1. Create an OpenSearch Index
1. Create a Knowledge Base
1. Create a data source and attach to the recently created Knowledge Base
1. Ingest data to your knowledge Base

### 3.1 Create Knowledge Base Role and Policies

Let's first create IAM policies to allow our Knowledge Base to access Bedrock Titan Embedding Foundation model, Amazon OpenSearch Serverless and the S3 bucket with the Knowledge Base Files.

Once the policies are ready, we will create the Knowledge Base role

In [9]:
# Create IAM policies for KB to invoke embedding model
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)

In [10]:
# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [11]:
kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [12]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'e3bb55cf-9b34-4089-9039-ade569a65b2d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 09 Feb 2025 09:57:12 GMT',
   'x-amzn-requestid': 'e3bb55cf-9b34-4089-9039-ade569a65b2d',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [13]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

'arn:aws:iam::174178623257:role/AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'

### 3.2 Create Vector Database

Firt of all we have to create a vector store. In this section we will use Amazon OpenSerach Serverless.

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.


In [14]:
# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)

In [15]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)

In [16]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

'arn:aws:iam::174178623257:user/Siddharth'

In [17]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)


In [18]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)
opensearch_collection_response

{'createCollectionDetail': {'arn': 'arn:aws:aoss:us-east-1:174178623257:collection/6p6prkia9g2l3vz8dq3d',
  'createdDate': 1739095035459,
  'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base',
  'id': '6p6prkia9g2l3vz8dq3d',
  'kmsKeyArn': 'auto',
  'lastModifiedDate': 1739095035459,
  'name': 'bd-kbc-us-east-1-174178623257',
  'standbyReplicas': 'DISABLED',
  'status': 'CREATING',
  'type': 'VECTORSEARCH'},
 'ResponseMetadata': {'RequestId': '88daa0aa-7ee4-4fe4-8fdd-485f94af457e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '88daa0aa-7ee4-4fe4-8fdd-485f94af457e',
   'date': 'Sun, 09 Feb 2025 09:57:15 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '394',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [19]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

'arn:aws:aoss:us-east-1:174178623257:collection/6p6prkia9g2l3vz8dq3d'

In [20]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host

Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...

Collection successfully created:
[{'arn': 'arn:aws:aoss:us-east-1:174178623257:collection/6p6prkia9g2l3vz8dq3d', 'collectionEndpoint': 'https://6p6prkia9g2l3vz8dq3d.us-east-1.aoss.amazonaws.com', 'createdDate': 1739095035459, 'dashboardEndpoint': 'https://6p6prkia9g2l3vz8dq3d.us-east-1.aoss.amazonaws.com/_dashboards', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base', 'id': '6p6prkia9g2l3vz8dq3d', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1739095263439, 'name': 'bd-kbc-us-east-1-174178623257', 'standbyReplicas': 'DISABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}]


'6p6prkia9g2l3vz8dq3d.us-east-1.aoss.amazonaws.com'

### 3.3 - Create OpenSearch Index

Let's now create a vector index to index our data


In [21]:
credentials = boto3.Session().get_credentials()
service = 'aoss'
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key,
    region, 
    service, 
    session_token=credentials.token
)

# Build the OpenSearch client
open_search_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(45)
index_body = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {}
    }
}

index_body["mappings"]["properties"][kb_vectorField] = {
    "type": "knn_vector",
    "dimension": 1536,
    "method": {
         "name": "hnsw",
         "engine": "faiss"
    },
}

index_body["mappings"]["properties"][kb_textField] = {
    "type": "text"
}

index_body["mappings"]["properties"][kb_metadataField] = {
    "type": "text"
}

# Create index
response = open_search_client.indices.create(kb_vector_index_name, body=index_body)
print('\nCreating index:')
print(response)

[2025-02-09 15:31:19,490] p57121 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2025-02-09 15:32:06,846] p57121 {base.py:258} INFO - PUT https://6p6prkia9g2l3vz8dq3d.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-index [status:200 request:2.321s]



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-index'}


### 3.5 - Create Knowledge Base
Now that we have the Vector database available in OpenSearch Serverless, let's create a Knowledge Base and associate it with the OpenSearch DB

In [22]:
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': collection_arn, 
        'fieldMapping': {
            'metadataField': kb_metadataField,
            'textField': kb_textField,
            'vectorField': kb_vectorField
        },
        'vectorIndexName': kb_vector_index_name
    },
    'type': 'OPENSEARCH_SERVERLESS'
}

In [25]:
# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1013',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 09 Feb 2025 10:09:57 GMT',
                                      'x-amz-apigw-id': 'FtjGXFIwoAMEfHg=',
                                      'x-amzn-requestid': '583958c5-c194-48e9-83fe-cf6bf125fdae',
                                      'x-amzn-trace-id': 'Root=1-67a87ef5-35ce614b5ef33858254b3ecf'},
                      'HTTPStatusCode': 202,
                      'RequestId': '583958c5-c194-48e9-83fe-cf6bf125fdae',
                      'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2025, 2, 9, 10, 9, 57, 273534, tzinfo=tzutc()),
                   'description': 'KB that contains the bedrock documentation',
                   'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:174178623257:knowledge-base/QVHUHU

In [26]:
# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': bucket_arn,
    'inclusionPrefixes': [kb_key]  
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'
}

knowledge_base_id = kb_obj["knowledgeBase"]["knowledgeBaseId"]
knowledge_base_arn = kb_obj["knowledgeBase"]["knowledgeBaseArn"]

chunking_strategy_configuration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# Create the data source
try:
    # ensure that the KB is created and available
    time.sleep(45)
    data_source_response = bedrock_agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name=data_source_name,
        description='DataSource for the bedrock documentation',
        dataSourceConfiguration=data_source_configuration,
        vectorIngestionConfiguration = {
            "chunkingConfiguration": chunking_strategy_configuration
        }
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '661',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 09 Feb 2025 10:10:47 GMT',
                                      'x-amz-apigw-id': 'FtjOPGAwoAMElIA=',
                                      'x-amzn-requestid': '39a1d2ad-5cf5-4979-b31a-002c4ce1b8e1',
                                      'x-amzn-trace-id': 'Root=1-67a87f27-68cb063062b88a914ceeb2ef'},
                      'HTTPStatusCode': 200,
                      'RequestId': '39a1d2ad-5cf5-4979-b31a-002c4ce1b8e1',
                      'RetryAttempts': 0},
 'dataSource': {'createdAt': datetime.datetime(2025, 2, 9, 10, 10, 47, 654810, tzinfo=tzutc()),
                'dataDeletionPolicy': 'DELETE',
                'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-docs-kb-agents-us-east-1-17417862325

### 3.6 - Start ingestion job

Once the Knowledge Base and Data Source are created, we can start the ingestion job. During the ingestion job, Knowledge Base will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case Amazon OpenSource Serverless.


In [27]:
# Start an ingestion job
data_source_id = data_source_response["dataSource"]["dataSourceId"]
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=knowledge_base_id, 
    dataSourceId=data_source_id
)


## 4. Create Agent

We will now create the Agent and associate the Knowledge Base to it. To do so we need to: 
1. Create Agent IAM role and policies
1. Create Agent
1. Associate Agent to Knowledge Base
1. Prepare Agent

### 4.1 - Create Agent IAM role and policies
First we need to create the agent policies that allow bedrock model invocation and Knowledge Base retrieval

In [28]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [29]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)


In [30]:

# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)


iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '68cfcb63-1936-46b3-bff0-daf687efc695',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 09 Feb 2025 10:11:07 GMT',
   'x-amzn-requestid': '68cfcb63-1936-46b3-bff0-daf687efc695',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### 4.2 - Create Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [31]:
# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent supporting Amazon Bedrock Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

Let's now store the agent id in a local variable to use it on the next steps

In [32]:
agent_id = response['agent']['agentId']
agent_id

'8IUUW4UFPS'

### 4.3 - Associate agent to the Knowledge Base
Next, we need to associate the agent created with the Knowledge Base for the Bedrock documentation

In [33]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Amazon Bedrock.',
    knowledgeBaseId=knowledge_base_id 
)

### 4.4 - Prepare Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [34]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '3ce25406-c738-42c5-ac05-5b353416a78f',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 09 Feb 2025 10:11:15 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3ce25406-c738-42c5-ac05-5b353416a78f',
   'x-amz-apigw-id': 'FtjSiE3EoAMEpqw=',
   'x-amzn-trace-id': 'Root=1-67a87f43-74b39af4010b0cdd1dbbefed'},
  'RetryAttempts': 0},
 'agentId': '8IUUW4UFPS',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 2, 9, 10, 11, 15, 309093, tzinfo=tzutc())}

## 5 - Testing Agent

Now that we have our agent, let's invoke it to test if it is providing correct information about Amazon Bedrock. To do so, let's first create an Agent Alias

In [35]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [36]:
agent_alias

{'ResponseMetadata': {'RequestId': '434a6637-4487-45cf-a63f-09bfb217bf2b',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 09 Feb 2025 10:11:47 GMT',
   'content-type': 'application/json',
   'content-length': '344',
   'connection': 'keep-alive',
   'x-amzn-requestid': '434a6637-4487-45cf-a63f-09bfb217bf2b',
   'x-amz-apigw-id': 'FtjXlGuZoAMElIA=',
   'x-amzn-trace-id': 'Root=1-67a87f63-6f6daa7606f394f6219200c5'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:174178623257:agent-alias/8IUUW4UFPS/SSOERRKTQM',
  'agentAliasId': 'SSOERRKTQM',
  'agentAliasName': 'bedrock-docs-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': '8IUUW4UFPS',
  'createdAt': datetime.datetime(2025, 2, 9, 10, 11, 47, 407018, tzinfo=tzutc()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2025, 2, 9, 10, 11, 47, 407018, tzinfo=tzutc())}}

Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and get the information from the Knowledge base

In [37]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How can I evaluate models on Bedrock?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))


[2025-02-09 15:42:21,514] p57121 {2021949151.py:19} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Sun, 09 Feb 2025 10:12:21 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'f5105130-c5d1-42b3-bb4c-a9ed987bc423'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'f5105130-c5d1-42b3-bb4c-a9ed987bc423',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x10c609790>,
 'contentType': 'application/json',
 'sessionId': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc'}


In [38]:

event_stream = agentResponse['completion']
try:
    traces1 = [] 
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
            traces1.append(event['trace'])
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-02-09 15:42:22,218] p57121 {2946046497.py:12} INFO - {
  "agentAliasId": "SSOERRKTQM",
  "agentId": "8IUUW4UFPS",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/8IUUW4UFPS/SSOERRKTQM"
    }
  ],
  "sessionId": "59151e68-e6ce-11ef-8611-da5cbc4eb4cc",
  "trace": {
    "orchestrationTrace": {
      "invocationInput": {
        "invocationType": "KNOWLEDGE_BASE",
        "knowledgeBaseLookupInput": {
          "knowledgeBaseId": "QVHUHUIFNN",
          "text": "How can I evaluate models on Bedrock?"
        },
        "traceId": "f5105130-c5d1-42b3-bb4c-a9ed987bc423-0"
      }
    }
  }
}
[2025-02-09 15:42:22,221] p57121 {2946046497.py:12} INFO - {
  "agentAliasId": "SSOERRKTQM",
  "agentId": "8IUUW4UFPS",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/8IUUW4UFPS/SSOERRKTQM"
    }
  ],
  "sessionId": "59151e68-e6ce-11ef-8611-da5cbc4eb

In [39]:
for trace in traces1:
    print(trace['trace']['orchestrationTrace'])
    print()

{'invocationInput': {'invocationType': 'KNOWLEDGE_BASE', 'knowledgeBaseLookupInput': {'knowledgeBaseId': 'QVHUHUIFNN', 'text': 'How can I evaluate models on Bedrock?'}, 'traceId': 'f5105130-c5d1-42b3-bb4c-a9ed987bc423-0'}}

{'observation': {'knowledgeBaseLookupOutput': {'retrievedReferences': [{'content': {'text': "Give the model evaluation job a name that describes the job. This name is shown in your model evaluation job list. The name must be unique in your AWS account in an AWS Region.     b. Description (Optional) ? Provide an optional description.     5. Then, choose Next.     6. On the Set up evaluation page provide the following.     Human worker based model evaluation jobs 296           https://console.aws.amazon.com/bedrock/homeAmazon Bedrock User Guide     a. Models ? You can choose up to two models you want to use in the model evaluation job.     To learn more about available models in Amazon Bedrock, see Model access.     b. (Optional) To change the inference configuration 

In [40]:
json_json = json.loads(traces1[2]['trace']['orchestrationTrace']['modelInvocationInput']['text'])

for i in json_json['messages']:
    print(i['role'])
    print(i['content'])
    print()

user
[{text=How can I evaluate models on Bedrock?}]

assistant
[{toolUse={input={searchQuery=How can I evaluate models on Bedrock?}, name=GET__x_amz_knowledgebase_QVHUHUIFNN__Search}}]

user
[{toolResult={toolUseId=toolu_bdrk_01KBprefetchWithoutLLM00, content=Here are search results in numbered order:<search_result>    <answer_part>    <text>        Understanding how the results of your model evaluation job that are saved in Amazon S3     Automated model evaluation job report cards (console)     In your model evaluation report card, you will see the total number of prompts in the dataset you provided or selected, and how many of those prompts received responses. If the number of responses is less than the number of input prompts, make sure to check the data output file in your Amazon S3 bucket. It is possible that the prompt caused an error with the model and there was no inference retrieved. Only responses from the model will be used in metric calculations.     Use the following proce

In [42]:
from converter import convert_to_ragas_messages

convert_to_ragas_messages(traces=traces1)

/Users/nexus/miniconda3/envs/tempo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[HumanMessage(content='How can I evaluate models on Bedrock?', metadata=None, type='human'),
 ToolMessage(content="Give the model evaluation job a name that describes the job. This name is shown in your model evaluation job list. The name must be unique in your AWS account in an AWS Region.     b. Description (Optional) ? Provide an optional description.     5. Then, choose Next.     6. On the Set up evaluation page provide the following.     Human worker based model evaluation jobs 296           https://console.aws.amazon.com/bedrock/homeAmazon Bedrock User Guide     a. Models ? You can choose up to two models you want to use in the model evaluation job.     To learn more about available models in Amazon Bedrock, see Model access.     b. (Optional) To change the inference configuration for the selected models choose update.     Changing the inference configuration changes the responses generated by the selected models. To learn more about the available inferences parameters, see Infer

In [43]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

To evaluate models on Amazon Bedrock, you can create an automatic model evaluation job. Here are the steps to follow:

1. Open the Amazon Bedrock console.
2. In the navigation pane, choose "Model evaluation."
3. In the "Build an evaluation" card, under "Automatic," choose "Create automatic evaluation."
4. Provide the following information:
   - Evaluation name: Give the model evaluation job a name that describes the job. This name must be unique in your AWS account in an AWS Region.
   - Description (Optional): Provide an optional description.
   - Models: Choose the model you want to use in the model evaluation job.
   - (Optional) To change the inference configuration, choose "update."
   - Task type: Choose the type of task you want the model to attempt to perform during the model evaluation job.
   - Metrics and datasets: Choose from the list of available metrics and built-in prompt datasets based on the task you select. You can also choose to use your own prompt dataset by enterin

In [44]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=True, end_session=False):
    if session_id is None:
        session_id:str = str(uuid.uuid1())

    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    agent_answer = ''
    event_stream = agentResponse['completion']
    try:
        traces = []
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer, traces
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
                traces.append(event['trace'])
            else:
                raise Exception("unexpected event.", event)
        return agent_answer, traces
    except Exception as e:
        raise Exception("unexpected event.", e)
    return agent_answer

In [45]:
agent_answer, traces2 = simple_agent_invoke("what is bedrock provisioned throughput?", agent_id, agent_alias_id, session_id)

[2025-02-09 15:42:53,689] p57121 {3800203880.py:13} INFO - None
[2025-02-09 15:42:53,868] p57121 {3800203880.py:28} INFO - {
  "agentAliasId": "SSOERRKTQM",
  "agentId": "8IUUW4UFPS",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/8IUUW4UFPS/SSOERRKTQM"
    }
  ],
  "sessionId": "59151e68-e6ce-11ef-8611-da5cbc4eb4cc",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 1024,
          "stopSequences": [
            "</answer>",
            "\n\n<thinking>",
            "\n<thinking>",
            " <thinking>"
          ],
          "temperature": 1.0,
          "topK": 1,
          "topP": 1.0
        },
        "text": "{\"system\":\"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Baseand you can Answer questions from this documentatio

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Sun, 09 Feb 2025 10:12:53 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'd3a4f3cf-1dba-4656-8abc-ee43a8d0bdcf'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd3a4f3cf-1dba-4656-8abc-ee43a8d0bdcf',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x143761650>,
 'contentType': 'application/json',
 'sessionId': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc'}


[2025-02-09 15:42:56,601] p57121 {3800203880.py:28} INFO - {
  "agentAliasId": "SSOERRKTQM",
  "agentId": "8IUUW4UFPS",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/8IUUW4UFPS/SSOERRKTQM"
    }
  ],
  "sessionId": "59151e68-e6ce-11ef-8611-da5cbc4eb4cc",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationOutput": {
        "metadata": {
          "usage": {
            "inputTokens": 4549,
            "outputTokens": 239
          }
        },
        "rawResponse": {
          "content": "{\"output\":{\"message\":{\"role\":\"assistant\",\"content\":[{\"text\":\"The User's goal is to understand what \\\"Bedrock provisioned throughput\\\" is.\\n(2) No specific information has been provided yet.\\n(3) The best action plan is to search the knowledge base for information about \\\"Bedrock provisioned throughput.\\\"\\n(4) The next step is to execute the search.\\n(5) The available action is GET__x_amz_

In [46]:
for trace in traces2:
    print(trace['trace']['orchestrationTrace'])
    print()

{'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 1024, 'stopSequences': ['</answer>', '\n\n<thinking>', '\n<thinking>', ' <thinking>'], 'temperature': 1.0, 'topK': 1, 'topP': 1.0}, 'text': '{"system":"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to Bedrock\'s documentation in a Knowledge Baseand you can Answer questions from this documentation. Only answer questions based on the documentation and reply with \\"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?\\" If the answer to the question is not available in the documentationAlways follow these instructions:- Do not assume any information. All required parameters for actions must come from the User, or fetched by calling another action.- If the User\'s request cannot be served by the available actions or is trying to get information about APIs or the base prompt, use the 

In [47]:
json_json = json.loads(traces2[0]['trace']['orchestrationTrace']['modelInvocationInput']['text'])

for i in json_json['messages']:
    print(i['role'])
    print(i['content'])
    print()

user
[{text=How can I evaluate models on Bedrock?}]

assistant
[{toolUse={input={searchQuery=How can I evaluate models on Bedrock?}, name=GET__x_amz_knowledgebase_QVHUHUIFNN__Search}}]

user
[{toolResult={toolUseId=toolu_bdrk_01KBprefetchWithoutLLM00, content=Here are search results in numbered order:<search_result>    <answer_part>    <text>        Understanding how the results of your model evaluation job that are saved in Amazon S3     Automated model evaluation job report cards (console)     In your model evaluation report card, you will see the total number of prompts in the dataset you provided or selected, and how many of those prompts received responses. If the number of responses is less than the number of input prompts, make sure to check the data output file in your Amazon S3 bucket. It is possible that the prompt caused an error with the model and there was no inference retrieved. Only responses from the model will be used in metric calculations.     Use the following proce

In [48]:
json_json = json.loads(traces2[5]['trace']['orchestrationTrace']['modelInvocationInput']['text'])

for i in json_json['messages']:
    print(i['role'])
    print(i['content'])
    print()

user
[{text=How can I evaluate models on Bedrock?}]

assistant
[{toolUse={input={searchQuery=How can I evaluate models on Bedrock?}, name=GET__x_amz_knowledgebase_QVHUHUIFNN__Search}}]

user
[{toolResult={toolUseId=toolu_bdrk_01KBprefetchWithoutLLM00, content=Here are search results in numbered order:<search_result>    <answer_part>    <text>        Understanding how the results of your model evaluation job that are saved in Amazon S3     Automated model evaluation job report cards (console)     In your model evaluation report card, you will see the total number of prompts in the dataset you provided or selected, and how many of those prompts received responses. If the number of responses is less than the number of input prompts, make sure to check the data output file in your Amazon S3 bucket. It is possible that the prompt caused an error with the model and there was no inference retrieved. Only responses from the model will be used in metric calculations.     Use the following proce

In [49]:
from converter import convert_to_ragas_messages

convert_to_ragas_messages(traces=traces2)

[HumanMessage(content='[{text=what is bedrock provisioned throughput?}]', metadata=None, type='human'),
 AIMessage(content='The User\'s goal is to understand what "Bedrock provisioned throughput" is.\n(2) No specific information has been provided yet.\n(3) The best action plan is to search the knowledge base for information about "Bedrock provisioned throughput."\n(4) The next step is to execute the search.\n(5) The available action is GET__x_amz_knowledgebase_QVHUHUIFNN__Search.\n(6) This action requires a searchQuery parameter, which should be "what is bedrock provisioned throughput?".\n(7) I have everything I need to execute the search.', metadata=None, type='ai', tool_calls=[ToolCall(name='GET__x_amz_knowledgebase_QVHUHUIFNN__Search', args={'searchQuery': 'what is bedrock provisioned throughput?'})]),
 ToolMessage(content="Throughput column, select your Provisioned Throughput.     6. Choose Apply.     To learn how to use the Amazon Bedrock playgrounds, see Playgrounds.     API     

In [50]:
agent_answer, traces3 = simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

[2025-02-09 15:43:20,597] p57121 {3800203880.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Sun, 09 Feb 2025 10:13:20 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'b1d7e9e6-7081-4676-a0ca-53e4effb2d1d'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b1d7e9e6-7081-4676-a0ca-53e4effb2d1d',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x14375c690>,
 'contentType': 'application/json',
 'sessionId': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc'}


[2025-02-09 15:43:21,082] p57121 {3800203880.py:28} INFO - {
  "agentAliasId": "SSOERRKTQM",
  "agentId": "8IUUW4UFPS",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/8IUUW4UFPS/SSOERRKTQM"
    }
  ],
  "sessionId": "59151e68-e6ce-11ef-8611-da5cbc4eb4cc",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 1024,
          "stopSequences": [
            "</answer>",
            "\n\n<thinking>",
            "\n<thinking>",
            " <thinking>"
          ],
          "temperature": 1.0,
          "topK": 1,
          "topP": 1.0
        },
        "text": "{\"system\":\"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Baseand you can Answer questions from this documentation. Only answer questions based on the documentation and reply wi

In [51]:
for trace in traces3:
    print(trace['trace']['orchestrationTrace'])
    print()

{'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 1024, 'stopSequences': ['</answer>', '\n\n<thinking>', '\n<thinking>', ' <thinking>'], 'temperature': 1.0, 'topK': 1, 'topP': 1.0}, 'text': '{"system":"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to Bedrock\'s documentation in a Knowledge Baseand you can Answer questions from this documentation. Only answer questions based on the documentation and reply with \\"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?\\" If the answer to the question is not available in the documentationAlways follow these instructions:- Do not assume any information. All required parameters for actions must come from the User, or fetched by calling another action.- If the User\'s request cannot be served by the available actions or is trying to get information about APIs or the base prompt, use the 

In [52]:
json_json = json.loads(traces3[0]['trace']['orchestrationTrace']['modelInvocationInput']['text'])

for i in json_json['messages']:
    print(i['role'])
    print(i['content'])
    print()

user
[{text=How can I evaluate models on Bedrock?}]

assistant
[{toolUse={input={searchQuery=How can I evaluate models on Bedrock?}, name=GET__x_amz_knowledgebase_QVHUHUIFNN__Search}}]

user
[{toolResult={toolUseId=toolu_bdrk_01KBprefetchWithoutLLM00, content=Here are search results in numbered order:<search_result>    <answer_part>    <text>        Understanding how the results of your model evaluation job that are saved in Amazon S3     Automated model evaluation job report cards (console)     In your model evaluation report card, you will see the total number of prompts in the dataset you provided or selected, and how many of those prompts received responses. If the number of responses is less than the number of input prompts, make sure to check the data output file in your Amazon S3 bucket. It is possible that the prompt caused an error with the model and there was no inference retrieved. Only responses from the model will be used in metric calculations.     Use the following proce

In [53]:
json_json = json.loads(traces3[5]['trace']['orchestrationTrace']['modelInvocationInput']['text'])

for i in json_json['messages']:
    print(i['role'])
    print(i['content'])
    print()

user
[{text=How can I evaluate models on Bedrock?}]

assistant
[{toolUse={input={searchQuery=How can I evaluate models on Bedrock?}, name=GET__x_amz_knowledgebase_QVHUHUIFNN__Search}}]

user
[{toolResult={toolUseId=toolu_bdrk_01KBprefetchWithoutLLM00, content=Here are search results in numbered order:<search_result>    <answer_part>    <text>        Understanding how the results of your model evaluation job that are saved in Amazon S3     Automated model evaluation job report cards (console)     In your model evaluation report card, you will see the total number of prompts in the dataset you provided or selected, and how many of those prompts received responses. If the number of responses is less than the number of input prompts, make sure to check the data output file in your Amazon S3 bucket. It is possible that the prompt caused an error with the model and there was no inference retrieved. Only responses from the model will be used in metric calculations.     Use the following proce

In [54]:
from converter import convert_to_ragas_messages

convert_to_ragas_messages(traces=traces3)

[HumanMessage(content='[{text=what are the components of a Bedrock Guardrail?}]', metadata=None, type='human'),
 AIMessage(content='The User\'s goal is to understand the components of a Bedrock Guardrail.\n(2) No specific information has been provided yet.\n(3) The best action plan is to search the knowledge base for information about the components of a Bedrock Guardrail.\n(4) The next step is to execute the search.\n(5) The available action is GET__x_amz_knowledgebase_QVHUHUIFNN__Search.\n(6) This action requires a searchQuery parameter, which should be "what are the components of a Bedrock Guardrail?".\n(7) I have everything I need to execute the search.', metadata=None, type='ai', tool_calls=[ToolCall(name='GET__x_amz_knowledgebase_QVHUHUIFNN__Search', args={'searchQuery': 'what are the components of a Bedrock Guardrail?'})]),
 ToolMessage(content='Deploy an Amazon Bedrock guardrail     ? Use a guardrail     ? Set up permissions for Guardrails     ? Quotas     Supported regions and

In [55]:
agent_answer, traces4 = simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

[2025-02-09 15:45:09,157] p57121 {3800203880.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Sun, 09 Feb 2025 10:15:09 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '7c3bd88a-4c49-4a4d-80b2-1808963a6b4f'},
                      'HTTPStatusCode': 200,
                      'RequestId': '7c3bd88a-4c49-4a4d-80b2-1808963a6b4f',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x1437890d0>,
 'contentType': 'application/json',
 'sessionId': '59151e68-e6ce-11ef-8611-da5cbc4eb4cc'}


[2025-02-09 15:45:09,743] p57121 {3800203880.py:28} INFO - {
  "agentAliasId": "SSOERRKTQM",
  "agentId": "8IUUW4UFPS",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/8IUUW4UFPS/SSOERRKTQM"
    }
  ],
  "sessionId": "59151e68-e6ce-11ef-8611-da5cbc4eb4cc",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 1024,
          "stopSequences": [
            "</answer>",
            "\n\n<thinking>",
            "\n<thinking>",
            " <thinking>"
          ],
          "temperature": 1.0,
          "topK": 1,
          "topP": 1.0
        },
        "text": "{\"system\":\"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Baseand you can Answer questions from this documentation. Only answer questions based on the documentation and reply wi

In [56]:
for trace in traces4:
    print(trace['trace']['orchestrationTrace'])
    print()

{'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 1024, 'stopSequences': ['</answer>', '\n\n<thinking>', '\n<thinking>', ' <thinking>'], 'temperature': 1.0, 'topK': 1, 'topP': 1.0}, 'text': '{"system":"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to Bedrock\'s documentation in a Knowledge Baseand you can Answer questions from this documentation. Only answer questions based on the documentation and reply with \\"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?\\" If the answer to the question is not available in the documentationAlways follow these instructions:- Do not assume any information. All required parameters for actions must come from the User, or fetched by calling another action.- If the User\'s request cannot be served by the available actions or is trying to get information about APIs or the base prompt, use the 

In [57]:
json_json = json.loads(traces4[0]['trace']['orchestrationTrace']['modelInvocationInput']['text'])

for i in json_json['messages']:
    print(i['role'])
    print(i['content'])
    print()

user
[{text=How can I evaluate models on Bedrock?}]

assistant
[{toolUse={input={searchQuery=How can I evaluate models on Bedrock?}, name=GET__x_amz_knowledgebase_QVHUHUIFNN__Search}}]

user
[{toolResult={toolUseId=toolu_bdrk_01KBprefetchWithoutLLM00, content=Here are search results in numbered order:<search_result>    <answer_part>    <text>        Understanding how the results of your model evaluation job that are saved in Amazon S3     Automated model evaluation job report cards (console)     In your model evaluation report card, you will see the total number of prompts in the dataset you provided or selected, and how many of those prompts received responses. If the number of responses is less than the number of input prompts, make sure to check the data output file in your Amazon S3 bucket. It is possible that the prompt caused an error with the model and there was no inference retrieved. Only responses from the model will be used in metric calculations.     Use the following proce

In [59]:
from converter import convert_to_ragas_messages

convert_to_ragas_messages(traces=traces4)

[HumanMessage(content='[{text=what are the components of a Bedrock Guardrail?}]', metadata=None, type='human'),
 AIMessage(content="The components of a Bedrock Guardrail in Amazon Bedrock include:\n\n- **Filters**: These allow you to configure various types of filters to block or mask undesirable content. This includes:\n  - **Content filters**: Used to block or mask content based on predefined criteria.\n  - **Denied topics**: Specific topics that you can define to be blocked.\n  - **Word filters**: Filters to block undesirable words, phrases, and profanity.\n  - **Sensitive information filters**: Filters to block or mask personally identifiable information (PII) and other sensitive patterns using regular expressions.\n\n- **Messages**: Customizable messages that are returned to users when their input or the model's response violates the policies defined in the guardrail.\n\n- **Versions**: A guardrail can have multiple versions. When you create a guardrail, a working draft is automat

## 6 - Clean up (Optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. delete agent alias
1. delete agent
1. delete the knowledge base
1. delete the OpenSearch Serverless vector store
1. empty created s3 bucket
1. delete s3 bucket


In [60]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [61]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [62]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'TWJTG2XZXWN58E22',
  'HostId': 'sR1yVeq6tcGAKe0i3XFnw9RmJpSpzMqBmBsnUJazGo5LsL0co/E9i5IiZn89H6qnYl+zFjMP5Ic=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'sR1yVeq6tcGAKe0i3XFnw9RmJpSpzMqBmBsnUJazGo5LsL0co/E9i5IiZn89H6qnYl+zFjMP5Ic=',
   'x-amz-request-id': 'TWJTG2XZXWN58E22',
   'date': 'Sun, 09 Feb 2025 10:23:11 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [63]:
# Delete IAM Roles and policies and Knowledge Base files
for policy in [
    agent_bedrock_policy, 
    agent_kb_schema_policy,
    kb_bedrock_policy,
    kb_aoss_policy,
    kb_s3_policy
]:
    response = iam_client.list_entities_for_policy(
        PolicyArn=policy['Policy']['Arn'],
        EntityFilter='Role'
    )

    for role in response['PolicyRoles']:
        iam_client.detach_role_policy(
            RoleName=role['RoleName'], 
            PolicyArn=policy['Policy']['Arn']
        )

    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
    )

    

for role_name in [
    agent_role_name, 
    kb_role_name
]:
    try: 
        iam_client.delete_role(
            RoleName=role_name
        )
    except Exception as e:
        print(e)
        print("couldn't delete role", role_name)
        
    
try:

    open_search_serverless_client.delete_collection(
        id=opensearch_collection_response["createCollectionDetail"]["id"]
    )

    open_search_serverless_client.delete_access_policy(
          name=kb_collection_name,
          type='data'
    )    

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='network'
    )   

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='encryption'
    )    
    bedrock_agent_client.delete_knowledge_base(
        knowledgeBaseId=knowledge_base_id
    )
except Exception as e:
    print(e)

## Conclusion

We have now experimented with using boto3 SDK to create, invoke and delete an agent having a single KB connected to it.
## Take aways

Adapt this notebook to create new agents for your application

## Thank You